In [1]:
%run 'scripts/my_utils.py'
%run 'scripts/gen_data.py' --confirm 'no'
%run 'scripts/dataset.py'
%run 'scripts/my_models.py'
%run 'scripts/function.py'

In [2]:

import torch
import gc , random
import numpy as np
from scripts import environ
from scripts import my_utils as my_utils
from scripts.function import *
from scripts.environ import get_config
from scripts.dataset import ModelData
config = get_config()

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
storage_loader = my_utils.versatile_storage(config['STORAGE_TYPE'])
process_name = 'train'
num_states = 3

class container():
    def __init__(self) -> None:
        pass

data = ModelData('day' , config)
ShareNames = container()
ShareNames.model_date_list = data.model_date_list
ShareNames.test_full_dates = data.test_full_dates
data2 = data

In [28]:
from scripts.my_models import *
net2 = MyTRA_LSTM(6,16,**config['tra_param'])
net2 = copy.deepcopy(MyTRA_LSTM(6,16,**config['tra_param']).base_model.cuda())

Sequential(
  (0): mod_parallel(
    (mod_list): ModuleList(
      (0): uni_rnn_encoder(
        (fc_in): Sequential()
        (fc_rnn): mod_lstm(
          (lstm): LSTM(6, 16, num_layers=2, batch_first=True, dropout=0.1)
        )
      )
    )
  )
  (1): mod_parallel(
    (mod_list): ModuleList(
      (0): uni_rnn_decoder(
        (fc_dec_mlp): Sequential(
          (0): Sequential(
            (0): Linear(in_features=16, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.01)
            (2): Dropout(p=0.1, inplace=False)
          )
          (1): Sequential(
            (0): Linear(in_features=16, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.01)
            (2): Dropout(p=0.1, inplace=False)
          )
        )
        (fc_hid_out): Linear(in_features=16, out_features=16, bias=True)
      )
    )
  )
)

In [30]:
from scripts.special.TRA import *
net = copy.deepcopy(TRA(nn.LSTM(6,16),16,3).cuda())

In [29]:
temp_model = MyLSTM(6 , hidden_dim = 16 , num_output = 1)
base_model = nn.Sequential(temp_model.encoder , temp_model.decoder)

net = copy.deepcopy(TRA(base_model , 16 , 3))

In [4]:
config['TRA_switch'] = True #affect 1.buffer includes hist_loss 2.seqlen includes labels of a big number
tra_num_states = config['buffer_param']['tra_num_states'] if config['TRA_switch'] else 1
data = data2
data2.create_dataloader('train' , 'train' , ShareNames.model_date_list[0] , config['MODEL_PARAM']['seqlens'][0])


In [5]:
net = net2.cuda()

In [9]:


loss_train , loss_valid , ic_train , ic_valid = [] , [] , [] , []
iter_train , iter_valid = data2.dataloaders['train'] , data2.dataloaders['valid']

net.train()
optimizer = torch.optim.Adam(net.parameters())

modifier = {
    'inputs': lambda inputs,batch,data:inputs,
    'metric': lambda metric,batch,data:metric,
    'update': lambda update,batch,data:None
}
if config['TRA_switch']:
    modifier['inputs'] = lambda inputs,batch,data:net.modifier_inputs(inputs,batch,data)
    modifier['metric'] = lambda metric,batch,data:net.modifier_metric(metric,batch,data)
    modifier['update'] = lambda update,batch,data:net.modifier_update(update,batch,data)

k = 'hist_loss'
a_train , a_valid = [] , []

for ii in range(10):
    l_train , l_valid = [] , []
    net.train()   
    for _ , batch_data in enumerate(iter_train):
        x , y , w , i = tuple([batch_data[k] for k in ['x','y','w','i']])
        # d , rw = data2.buffer[k] , data2.seqs[k]
        inputs = modifier['inputs'](x , batch_data , data2)
        optimizer.zero_grad()
        
        pred , hidden = net(inputs)
        metric = {'loss':-pearson(pred , y) , 'score':pearson(pred , y).item()}
        metric = modifier['metric'](metric , batch_data , data2)
        
        metric['loss'].backward()
        optimizer.step()
        
        modifier['update'](None , batch_data , data2)

        l_train.append(metric['loss'].item())

    net.eval()     
    for i , batch_data in enumerate(iter_valid):
        x , y , w , i = tuple([batch_data[k] for k in ['x','y','w','i']])
        inputs = modifier['inputs'](x , batch_data , data2)
        pred , _ = net(inputs)
        metric = {'loss':-pearson(pred , y) , 'score':pearson(pred , y).item()}
        metric = modifier['metric'](metric , batch_data , data2)
        modifier['update'](None , batch_data , data2)
        l_valid.append(metric['score'])

    print(-np.mean(l_train) , np.mean(l_valid))



torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([10000, 8]) torch.Size([10000, 16])
torch.Size([10000, 16]) torch.Size([10000, 3]) torch.Size([1

In [13]:
net = net.requires_grad_(False)

In [19]:
import copy
for k in net.parameters():
    a = copy.deepcopy(k)
    print(k)

Parameter containing:
tensor([[-2.0756e-02, -2.3668e-01,  2.1613e-01, -1.3128e-01, -5.9777e-02,
         -2.4532e-02],
        [-1.1829e-01,  9.9294e-02,  3.0256e-01, -3.4906e-01, -1.5747e-01,
          1.3617e-01],
        [-2.4003e-01,  1.0682e-03,  1.8857e-01,  1.9728e-01,  8.9667e-02,
         -1.4220e-01],
        [ 3.1594e-01, -8.0494e-02, -1.5301e-02,  1.8239e-01, -2.7472e-01,
          2.0753e-01],
        [-2.0601e-01, -1.4605e-01,  5.0568e-02,  9.3283e-03, -1.5429e-02,
         -1.4979e-01],
        [ 2.9839e-02,  7.6466e-02,  4.3698e-02,  6.9855e-02,  4.9223e-01,
         -2.7686e-02],
        [-1.3446e-01,  2.4175e-01,  1.3917e-01, -1.1043e-01,  3.1870e-01,
         -2.5925e-01],
        [-1.3457e-01,  3.7840e-02, -1.9103e-01, -9.0541e-02, -1.6080e-01,
         -2.2518e-01],
        [-1.4282e-01, -5.7854e-02,  1.4515e-01,  2.1905e-02, -1.3699e-01,
          1.6835e-01],
        [-2.9060e-02, -2.1600e-01,  2.3905e-02, -7.5458e-02, -1.2462e-01,
          1.0851e-01],
        

In [16]:
k

'hist_loss'

In [11]:
inputs[0].shape , inputs[1].shape

(torch.Size([10000, 40, 6]), torch.Size([10000, 40, 3]))

In [20]:
hidden = net.base_model(inputs[0])[0].shape
hidden
# hidden[:,-1].shape
#net.predictors(hidden[:,-1])

torch.Size([10000, 16])

In [18]:
n = torch.tensor(np.array([True,False,True,True,False,True]))
for j in torch.utils.data.DataLoader(np.arange(len(n)) , batch_size = 2):
    print(j[n[j]])
torch.where(n)[0]

tensor([0])
tensor([2, 3])
tensor([5])


tensor([0, 2, 3, 5])

In [ ]:
'startwith'.startswith